In [1]:
import configparser
import os
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.functions import udf, col, lit
from pyspark.sql.types import MapType, StringType
from collections import OrderedDict
import pandas as pd
import numpy as np

In [2]:
config = configparser.ConfigParser()
config.read("capstone.cfg")

input_data_dir = config["PATH"]["HDFS_DOWNLOAD_DIR"]
local_data_dir = config["PATH"]["LOCAL_DOWNLOAD_DIR"]
output_path =  config["PATH"]["STAGE_DATA_DIR"]


In [3]:
# !!! must use python3 kernel
def create_spark_session():
    spark = SparkSession \
        .builder \
        .appName("covid_DB") \
        .getOrCreate()
    
    return spark

In [4]:
@udf(MapType( StringType(), StringType()))
def ParseGazetteerUDF(line):
    l_str = line.split()
    l = len(l_str)
    l_headers = 10
    n_words = l - l_headers + 1
    county = " ".join( l_str[3:3+n_words] )

    return{
        "state": l_str[0],
        "county" : county ,        
        "fips" : l_str[1], 
        "latitude" : l_str[-2], 
        "longitude" : l_str[-1] 
    }

In [5]:
# def main():
spark = create_spark_session()

nyt_covid = "us-counties.csv"
covid_daily_perfips = spark.read.csv( os.path.join(input_data_dir, "COVID",nyt_covid), header = True)
#covid_daily_perfips.printSchema()

# load gazetteer (geographic coordinates for all county fips)
fields = OrderedDict( [
    ( "state" , "string"),
    ("county" , "string"),
    ("fips" , "int"),
    ( "latitude" , "float"), 
    ("longitude" , "float") 
] )

exprs = [ f"CAST(parsed['{field}'] AS {fld_type}) AS {field}" for field, fld_type in fields.items() ]

gazetteer = spark.read.csv(os.path.join(input_data_dir, "2020_Gaz_counties_national.txt"))\
.withColumn("parsed", ParseGazetteerUDF("_c0")).selectExpr( *exprs)

#gazetteer.printSchema()


In [12]:
nyt_locations = covid_daily_perfips.select("state", "county", "fips").distinct()
fips_multi_locations = nyt_locations.groupby("fips").count().where("count > 1")
fips_multi_locations.show(10)
nyt_locations_with_fips = nyt_locations.where(col("fips").isNotNull())

+----+-----+
|fips|count|
+----+-----+
|null|   56|
+----+-----+



In [7]:
nyt_locations_with_fips.count()

3218

In [13]:
nyt_locations_with_geography = nyt_locations_with_fips.alias("a").join( gazetteer.alias("b"), ["fips"])\
    .select("a.*", "b.latitude", "b.longitude" )

In [14]:
nyt_notmatched = nyt_locations.join(nyt_locations_with_geography, ["fips"], how = "left_anti")

In [17]:
df_nyt_notmatched = nyt_notmatched.toPandas()

In [15]:
states_gps = pd.read_csv( os.path.join(local_data_dir, "US_states_GPS.csv"), sep = ",")
states_gps.columns = states_gps.columns.str.lower()
states_gps.loc[states_gps["state"] == "Washington State", "state"]= "Washington"

In [18]:
gps_no_county = df_nyt_notmatched.loc[df_nyt_notmatched["county"] =="Unknown"].merge( states_gps, how = "left", on = "state")
gps_no_county[ gps_no_county["latitude"].isna() | gps_no_county["longitude"].isna()]

,fips,state,county,latitude,longitude
2,None,Guam,Unknown,NaN,NaN
13,None,Virgin Islands,Unknown,NaN,NaN
35,None,Puerto Rico,Unknown,NaN,NaN
38,None,Northern Mariana Islands,Unknown,NaN,NaN


In [19]:
gps_cities = df_nyt_notmatched[ df_nyt_notmatched["county"] != "Unknown"].copy()

In [20]:
# GPS of unincorporated territories
gps_no_county.loc[ (gps_no_county["county"] == "Unknown") & (gps_no_county["state"]== "Puerto Rico") , "latitude"] = 18.2223
gps_no_county.loc[ (gps_no_county["county"] == "Unknown") & (gps_no_county["state"]=="Puerto Rico"), "longitude"] = -66.4303
gps_no_county.loc[ (gps_no_county["county"] == "Unknown") & (gps_no_county["state"]=="Virgin Islands"), "latitude"] = 18.34
gps_no_county.loc[ (gps_no_county["county"] == "Unknown") & (gps_no_county["state"]=="Virgin Islands"), "longitude"] = -64.90
gps_no_county.loc[ (gps_no_county["county"] == "Unknown") & (gps_no_county["state"]=="Guam"), "latitude"] = 13.4440
gps_no_county.loc[ (gps_no_county["county"] == "Unknown") & (gps_no_county["state"]=="Guam"), "longitude"] = 144.7671
# GPS coordinates of Saipan 15°11′N 145°45′E
gps_no_county.loc[ (gps_no_county["county"] == "Unknown") & (gps_no_county["state"]=="Northern Mariana Islands"), "latitude"] = 15.16
gps_no_county.loc[ (gps_no_county["county"] == "Unknown") & (gps_no_county["state"]=="Northern Mariana Islands"), "longitude"] = 145.7


In [21]:
gps_cities["latitude"] = np.nan
gps_cities["longitude"] = np.nan
# Cities and metropolitan areas
gps_cities.loc[ gps_cities["county"] == "New York City", "latitude"] = 40.712740
gps_cities.loc[ gps_cities["county"] == "New York City", "longitude"] = -74.005974
gps_cities.loc[ gps_cities["county"] == "Kansas City", "latitude"] = 39.099724
gps_cities.loc[ gps_cities["county"] == "Kansas City", "longitude"] = -94.578331
gps_cities.loc[ gps_cities["county"] == "Joplin", "latitude"] = 37.0842
gps_cities.loc[ gps_cities["county"] == "Joplin", "longitude"] = -94.5133
# FIPS unknown in Census Bureau gazetteer
gps_cities.loc[ gps_cities["county"] == "St. Croix", "latitude"] =17.73
gps_cities.loc[ gps_cities["county"] == "St. Croix", "longitude"] = -64.78
# GPS for Cruz Bay (main city) 18.329936603847486, -64.79413842601294
gps_cities.loc[ gps_cities["county"] == "St. John", "latitude"] = 18.33
gps_cities.loc[ gps_cities["county"] == "St. John", "longitude"] = -64.794
# GPS for Charlotte Amalie (main city) 18.341684050871354, -64.93175706594377
gps_cities.loc[ gps_cities["county"] == "St. Thomas", "latitude"] = 18.34
gps_cities.loc[ gps_cities["county"] == "St. Thomas", "longitude"] = -64.93
gps_cities.loc[ gps_cities["county"] == "Valdez-Cordova Census Area", "latitude"] = 61.129050
# GPS of Valdez
gps_cities.loc[ gps_cities["county"] == "Valdez-Cordova Census Area", "longitude"] = -146.360130
# Saipan 15.216501472234945, 145.72103373823464
gps_cities.loc[ gps_cities["county"] == "Saipan", "latitude"] = 15.27
gps_cities.loc[ gps_cities["county"] == "Saipan", "longitude"] = 145.72
# Tinian 14.978910978711687, 145.63629283555494
gps_cities.loc[ gps_cities["county"] == "Tinian", "latitude"] = 14.98
gps_cities.loc[ gps_cities["county"] == "Tinian", "longitude"] = 145.636

In [22]:
df_fixed_gps = pd.concat( [gps_no_county, gps_cities] )
len(df_fixed_gps)

64

In [23]:
fixed_gps = spark.createDataFrame(df_fixed_gps)
fixed_gps.printSchema()

root
 |-- fips: string (nullable = true)
 |-- state: string (nullable = true)
 |-- county: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [24]:
nyt_locations_with_geography.printSchema()

root
 |-- fips: string (nullable = true)
 |-- state: string (nullable = true)
 |-- county: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)



In [25]:
nyt_geography = nyt_locations_with_geography.union(fixed_gps)

In [26]:
nyt_geography.count()

3274

In [27]:
nyt_geography = nyt_geography.where( ( ~ F.isnan("latitude") ) | (~ F.isnan("longitude")) )

nyt_geography = nyt_geography.withColumn("location_id", F.monotonically_increasing_id())

In [28]:
nyt_geography.count()

3272

In [29]:
nyt_geography.printSchema()

root
 |-- fips: string (nullable = true)
 |-- state: string (nullable = true)
 |-- county: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- location_id: long (nullable = false)



In [24]:
out_path = os.path.join(output_path, "nyt_locations_geography")
nyt_geography.repartition("state")\
.write.format("parquet")\
.partitionBy("state").mode("overwrite").save(out_path)